In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import os
import joblib
from typing import Literal

In [2]:
df = pd.read_csv("document/restaurant sale/restaurant_sale.csv", index_col="id")

In [3]:
def fitModel(model, X_train, y_train):
    
    return model.fit(X_train, y_train)


def trainModel(X_train, Y_train):
    hgb = fitModel(GradientBoostingRegressor(n_estimators=100,
                                            max_depth=5,
                                            min_samples_leaf=5,
                                            learning_rate=0.1),
                   X_train,
                   Y_train)
    
    return hgb

In [4]:
# Convert categorical columns to dummy variables
df = pd.get_dummies(df, columns=['category'], drop_first=True)

# Split the data into features (X) and target (y)
X = df.drop('num_orders', axis=1)
y = df['num_orders']

In [5]:
model = trainModel(X, y)

joblib.dump(model, 'model/restaurant_sale_predictive.pkl')

['model/restaurant_sale_predictive.pkl']

In [6]:
"R square: ", model.score(X, y)

('R square: ', 0.388459010038383)

In [7]:
df.head()

,week,base_price,num_orders,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters
id,,,,,,,,,,,,,,,,
1379560,1,152.29,177,False,False,False,False,False,False,False,False,False,False,False,False,False
1466964,1,135.83,270,False,False,False,False,False,False,False,False,False,False,False,False,False
1346989,1,135.86,189,False,False,False,False,False,False,False,False,False,False,False,False,False
1338232,1,437.53,54,False,False,False,False,False,False,False,False,False,False,False,False,False
1448490,1,242.50,40,False,False,False,False,False,False,False,False,False,False,False,False,False


In [8]:
df.columns

Index(['week', 'base_price', 'num_orders', 'category_Biryani',
       'category_Desert', 'category_Extras', 'category_Fish',
       'category_Other Snacks', 'category_Pasta', 'category_Pizza',
       'category_Rice Bowl', 'category_Salad', 'category_Sandwich',
       'category_Seafood', 'category_Soup', 'category_Starters'],
      dtype='object')

In [19]:
import pandas as pd
import os
import joblib
from typing import Literal

def restaurant_sale_project(
    base_price:float, 
    category: Literal[
        'Beverages',
        'Biryani',
        'Desert',
        'Extras',
        'Fish',
        'Other Snacks',
        'Pasta',
        'Pizza',
        'Rice Bowl',
        'Salad',
        'Sandwich',
        'Seafood',
        'Soup',
        'Starters'
        ],  
    week:int|list=[1, 52]):

    def getModel():
        
        pkl_dir = os.path.abspath('model')
        
        model_path = os.path.join(pkl_dir, 'restaurant_sale_predictive.pkl')

        model = joblib.load(model_path)

        return model


    def runModel(X_test):

        model = getModel()
        Y_pred = model.predict(X_test)
        
        return model, Y_pred
    
    week = [week] if isinstance(week, int) else week
    
    result = {}
    
    for week in week:
        data = {
            "week": week,
            "base_price": base_price,
        }
        
        ca_key = [ 'category_Biryani',
        'category_Desert', 'category_Extras', 'category_Fish',
        'category_Other Snacks', 'category_Pasta', 'category_Pizza',
        'category_Rice Bowl', 'category_Salad', 'category_Sandwich',
        'category_Seafood', 'category_Soup', 'category_Starters']
        
        for k in ca_key:
            data[k] = category==k.split("_")[-1]
            
        data = pd.DataFrame([data])
        
        _, y = runModel(data)
        
        result["week " + str(week)] = round(y[0], 1)
        
    return result
